In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
# Add the directory to the Python path
sys.path.append(r'C:\Users\noori\Desktop\e-BRDF\Experimentell\2025_Code\MWIR-Daten')
from brdfAnalysisLib import *
import pickle
import temp_2_radiance as t2r
from joblib import Parallel, delayed

In [9]:
fnames0 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Aluminium\h5\*.h5"))
fnames1 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Alu_Eloxiert\h5\*.h5"))
fnames2 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Aluminium_Sandgestrahlt\h5\*.h5"))
fnames3 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Stahl_Sandgestrahlt\h5\*.h5"))
fnames4 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Referenzlack_Rau\h5\*.h5"))
fnames5 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Referenzlack_Glatt\h5\*.h5"))

fnames6 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Aluminium_Geschliffen\h5\*.h5"))
fnames7 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Edelstahl\h5\*.h5"))
fnames8 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\EdelstahlGebuerstet\h5\*.h5"))
fnames9 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Kupfer\h5\*.h5"))
fnames10 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Messing\h5\*.h5"))
fnames11 = sorted(glob.glob(r"\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Surfaces\Stahl_Verzinkt\h5\*.h5"))
names = ['Alu', 'Alu eloxiert', 'Alu sandgestrahlt', 'Stahl sandgestrahlt', 'Referenzlack (Rau)', 'Referenzlack (Glatt)', 'Aluminium geschliffen', 'Edelstahl', 'Edelstahl gebürstet', 'Kupfer', 'Messing', 'Verzinkter Stahl']

fnames_all = [fnames0, fnames1, fnames2, fnames3, fnames4, fnames5, fnames6, fnames7, fnames8, fnames9, fnames10, fnames11]

with open(r'\\mrt-pc1\mrt\Forschung\03_Projekte_laufende\517_e-BRDF_WiVoPro\11_Projektinhalte\05_Messungen\JSSS_MWIR_BRDF\Processed\center_rois.pickle', 'rb') as handle:
    center_rois = pickle.load(handle)
    
thetas_i = np.arange(1, 91, 1)[::-1]
thetas_i_rad = np.deg2rad(thetas_i)
thetas_o = np.array([15, 20, 30, 40, 45, 50, 60, 70, 80, 85])
thetas_o_rad = np.deg2rad(thetas_o)

def get_radiances(img_, x_center, y_center):
    img = img_.copy()
    mask = create_circular_mask(img.shape[0], img.shape[1], (x_center, y_center), radius=12)
    return np.mean(img[mask])

Reflections= {}

In [10]:
center_rois.keys()

dict_keys(['Alu', 'Edelstahl gebürstet', 'Referenzlack (Glatt)', 'Alu sandgestrahlt', 'Stahl sandgestrahlt', 'Verzinkter Stahl', 'Alu eloxiert', 'Alu geschliffen', 'Edelstahl', 'Messing', 'Referenzlack (Rau)', 'Kupfer'])

In [11]:
for j in range(len(names)):
    fnames = fnames_all[j]
    Ldiff = []
    
    if names[j] == 'Aluminium geschliffen':
        x_center, y_center, _ = center_rois['Alu geschliffen']
    else:
        x_center, y_center, _ = center_rois[names[j]]
    for i in range(len(thetas_o)):
        fname = fnames[i]
        hf_raw = h5py.File(fname, mode='r')
        hf = hf_raw['Radiancedata']
        imgs = np.asarray(hf).copy()
        hf_raw.close()
        delayed_calls = [delayed(get_radiances)(img, x_center, y_center) for img in imgs]
        L = Parallel(n_jobs=-1)(delayed_calls)
        L = np.asarray(L)
        del imgs
        Ldiff.append(np.abs(L[:90]-L[90:]))

    Reflections[names[j]] = {
        'L_o': np.array(Ldiff)
    }

In [13]:
with open('data.pkl', 'wb') as handle:
    pickle.dump(Reflections, handle, protocol=pickle.HIGHEST_PROTOCOL)